In [57]:
import os 
import cv2   
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
import numpy as np  
import matplotlib.pyplot as plt
import random
import tensorflow as tf
import tensorflow.keras.layers as Layers
import tensorflow.keras.activations as Actications
import tensorflow.keras.models as Models
import tensorflow.keras.optimizers as Optimizer
import tensorflow.keras.metrics as Metrics
import tensorflow.keras.utils as Utils
from IPython.display import SVG
from scipy.stats import mode


In [58]:
#Load the data
def get_images(directory):
    Images = []
    Labels = []  # 0 for Building , 1 for forest, 2 for glacier, 3 for mountain, 4 for Sea , 5 for Street
    class_labels = {'buildings': 0,
                    'forest' : 1,
                    'glacier' : 2,
                    'mountain' : 3,
                    'sea' : 4,
                    'street' : 5
                    }
    for label in os.listdir(directory):
        
        for image_file in os.listdir(directory+label): #Extracting the file name of the image from Class Label folder
            image = cv2.imread(directory+label+r'/'+image_file) #Reading the image (OpenCV)
            image = cv2.resize(image,(150,150)) #Resize the image, Some images are different sizes. (Resizing is very Important)
            Images.append(image)
            Labels.append(class_labels[label])
    
    return shuffle(Images,Labels) #Shuffle the dataset you just prepared.



In [59]:
Images, Labels = get_images('./seg_train/') #Extract the training images from the folders.

X_train = np.array(Images, dtype = 'float32') #converting the list of images to numpy array.
y_train = np.array(Labels, dtype = 'int32')


seg_x = []
seg_y = []

for image in X_train:
    seg_x.append(image[:50,:50,:])
    seg_x.append(image[:50,50:100,:])
    seg_x.append(image[:50,100:150,:])
    seg_x.append(image[50:100,:50,:])
    seg_x.append(image[50:100,50:100,:])
    seg_x.append(image[50:100,100:150,:])
    seg_x.append(image[100:,:50,:])
    seg_x.append(image[100:,50:100,:])
    seg_x.append(image[100:,100:150,:])

for label in y_train:
    seg_y.append(label)
    seg_y.append(label)
    seg_y.append(label)
    seg_y.append(label)
    seg_y.append(label)
    seg_y.append(label)
    seg_y.append(label)
    seg_y.append(label)
    seg_y.append(label)
  
seg_x = np.array(seg_x)
seg_y = np.array(seg_y)
seg_x = seg_x/(seg_x.max()) #Scale data

In [60]:
#Pull 1st 9000 segments (1k full images)
sx_train = seg_x[:9000]
sy_train = seg_y[:9000]

In [61]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu', input_shape = (50, 50, 3)), # the nn will learn the good filter to use
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(6, activation=tf.nn.softmax)
])


In [62]:
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

In [63]:
history = model.fit(sx_train, sy_train, batch_size=128, epochs=5, validation_split = 0.2)

Train on 7200 samples, validate on 1800 samples
Epoch 1/5
7200/7200 [==============================] - 206s 29ms/step - loss: 1.5282 - acc: 0.3568 - val_loss: 1.4574 - val_acc: 0.4183
Epoch 2/5
7200/7200 [==============================] - 217s 30ms/step - loss: 1.3470 - acc: 0.4537 - val_loss: 1.4091 - val_acc: 0.4289
Epoch 3/5
7200/7200 [==============================] - 223s 31ms/step - loss: 1.2528 - acc: 0.4918 - val_loss: 1.4567 - val_acc: 0.4233
Epoch 4/5
7200/7200 [==============================] - 222s 31ms/step - loss: 1.1962 - acc: 0.5276 - val_loss: 1.3331 - val_acc: 0.4772
Epoch 5/5
7200/7200 [==============================] - 224s 31ms/step - loss: 1.1197 - acc: 0.5568 - val_loss: 1.3310 - val_acc: 0.4706


In [64]:
predictions = model.predict(seg_x[9009:9018])
pred_labels = np.argmax(predictions, axis = 1)
pred_labels

array([2, 2, 3, 3, 3, 4, 3, 1, 1], dtype=int64)

In [65]:
pred = mode(pred_labels)
actual = mode(seg_y[9009:9018])

In [66]:
##Draw back of the mode is that it settles ties with the lower number
preds=[]
actuals=[]
for i in range(0,1000):
    lower = 9000+i*9
    upper = lower+9
    prediction = model.predict(seg_x[lower:upper])
    pred_label = np.argmax(prediction, axis = 1)
    preds.append(mode(pred_label,axis=None)[0][0])
    actuals.append(mode(seg_y[lower:upper],axis=None)[0][0])

In [67]:
print(preds)

[0, 3, 3, 4, 2, 4, 3, 0, 2, 4, 3, 4, 3, 2, 0, 2, 0, 4, 4, 3, 4, 4, 4, 0, 3, 1, 0, 1, 0, 3, 1, 0, 2, 1, 2, 0, 4, 1, 0, 1, 3, 0, 2, 0, 3, 3, 1, 0, 5, 1, 4, 1, 2, 2, 0, 1, 1, 2, 1, 0, 5, 1, 2, 3, 0, 3, 4, 0, 1, 3, 0, 4, 0, 4, 3, 4, 0, 0, 0, 4, 3, 0, 0, 3, 4, 4, 3, 1, 2, 0, 4, 0, 0, 1, 4, 3, 4, 4, 4, 1, 2, 2, 2, 3, 4, 1, 0, 0, 3, 1, 3, 2, 3, 3, 4, 5, 3, 5, 2, 0, 4, 4, 4, 2, 1, 1, 1, 4, 1, 2, 2, 0, 1, 4, 4, 3, 4, 4, 1, 1, 4, 0, 1, 0, 4, 1, 4, 3, 0, 3, 3, 2, 0, 4, 3, 4, 0, 1, 3, 4, 0, 1, 4, 0, 0, 4, 0, 0, 2, 0, 0, 1, 4, 1, 0, 4, 3, 3, 3, 4, 0, 0, 2, 0, 3, 3, 2, 3, 2, 0, 0, 4, 3, 3, 1, 3, 1, 4, 4, 3, 1, 1, 4, 0, 1, 3, 4, 0, 0, 1, 0, 2, 3, 4, 0, 3, 4, 3, 0, 4, 4, 1, 4, 1, 3, 2, 2, 3, 1, 3, 1, 4, 4, 0, 0, 4, 2, 4, 3, 4, 1, 0, 1, 3, 3, 2, 1, 1, 1, 3, 0, 4, 0, 4, 0, 3, 3, 3, 4, 1, 1, 4, 0, 0, 2, 1, 4, 0, 4, 4, 1, 0, 4, 2, 0, 5, 2, 3, 0, 2, 0, 0, 0, 4, 2, 0, 0, 1, 4, 1, 4, 0, 0, 4, 3, 2, 0, 4, 0, 4, 0, 3, 0, 0, 2, 3, 0, 0, 1, 2, 4, 4, 4, 1, 1, 1, 1, 0, 0, 1, 1, 2, 0, 1, 2, 4, 3, 5, 4, 0, 3, 4, 4, 

In [68]:
print(actuals)

[5, 3, 2, 4, 3, 4, 2, 0, 2, 4, 4, 4, 2, 2, 5, 2, 0, 3, 2, 3, 4, 0, 4, 0, 3, 1, 5, 1, 0, 3, 1, 0, 3, 1, 4, 0, 5, 1, 0, 1, 2, 0, 2, 5, 3, 2, 1, 0, 5, 1, 3, 1, 2, 2, 0, 1, 1, 2, 1, 0, 5, 1, 1, 3, 5, 3, 4, 0, 1, 0, 5, 4, 0, 3, 3, 4, 0, 5, 0, 3, 3, 5, 0, 1, 5, 4, 3, 0, 0, 4, 3, 5, 5, 1, 4, 5, 2, 4, 4, 1, 2, 2, 2, 3, 5, 1, 1, 5, 3, 3, 3, 2, 2, 3, 4, 5, 0, 0, 3, 0, 2, 5, 3, 4, 1, 1, 5, 4, 1, 3, 2, 0, 1, 2, 4, 2, 0, 2, 0, 0, 4, 0, 1, 5, 3, 1, 4, 2, 5, 2, 3, 3, 5, 2, 3, 4, 0, 1, 4, 4, 5, 1, 4, 5, 3, 4, 0, 0, 2, 5, 5, 1, 2, 0, 5, 2, 3, 3, 3, 2, 5, 0, 2, 0, 2, 4, 3, 1, 4, 0, 0, 4, 3, 3, 1, 2, 1, 4, 4, 3, 1, 2, 5, 5, 1, 3, 3, 5, 1, 1, 0, 2, 1, 4, 0, 2, 4, 3, 0, 4, 2, 1, 5, 1, 3, 2, 2, 3, 1, 3, 1, 3, 3, 5, 0, 1, 2, 3, 3, 0, 1, 0, 1, 2, 3, 4, 0, 1, 1, 3, 5, 4, 5, 1, 1, 2, 5, 4, 0, 1, 1, 4, 0, 0, 4, 1, 3, 0, 0, 4, 1, 0, 2, 0, 5, 0, 2, 5, 0, 3, 0, 0, 5, 4, 3, 5, 0, 1, 4, 1, 4, 1, 5, 4, 3, 2, 0, 5, 5, 4, 0, 4, 5, 1, 3, 0, 5, 0, 1, 2, 2, 4, 3, 1, 1, 1, 3, 5, 1, 1, 1, 2, 5, 1, 3, 3, 3, 5, 4, 0, 4, 4, 2, 

In [69]:
from sklearn.metrics import confusion_matrix
conf_mx=confusion_matrix(actuals, preds)
print(conf_mx)

[[114   8   5   5  16   5]
 [ 13 145   2   5   4   1]
 [  2   5  84  27  40   0]
 [  3   7  24 110  40   0]
 [  4   0  16  29 123   0]
 [111   6   1   4  22  19]]


In [71]:
print((114   +145   +84  +110  +123   +19)/len(preds))

0.595


In [72]:
'''Next steps:
    Try taking the top prediction from the 6x9 group
    sum across the 9 before argmax to get weighted votes
'''
predictions = model.predict(seg_x[9009:9018])
pred_labels = np.argmax(predictions, axis = 1)
#pred_labels
print(predictions)

[[1.4599060e-01 4.4217429e-04 3.9891428e-01 2.5032818e-01 1.8092330e-01
  2.3401430e-02]
 [2.7461177e-01 4.5100236e-04 4.1144851e-01 1.9702666e-01 8.3744012e-02
  3.2718107e-02]
 [1.4049302e-01 4.6960032e-04 2.4844940e-01 3.9476058e-01 1.9745257e-01
  1.8374790e-02]
 [2.4373454e-01 1.3571661e-02 5.3201862e-02 3.0520022e-01 2.9287875e-01
  9.1413036e-02]
 [1.5745896e-01 7.8864582e-03 7.0366293e-02 4.8670584e-01 2.2391257e-01
  5.3669840e-02]
 [3.3314773e-01 8.1089009e-03 3.8101573e-02 1.7588286e-01 3.6456090e-01
  8.0198035e-02]
 [4.5127485e-02 3.8967785e-01 5.1210381e-02 4.0297326e-01 9.9205174e-02
  1.1805982e-02]
 [3.2584459e-02 6.4848334e-01 2.7387638e-02 2.3980151e-01 4.3518163e-02
  8.2249027e-03]
 [4.0494408e-02 4.4490665e-01 3.9558809e-02 3.4717286e-01 1.1849638e-01
  9.3708672e-03]]


In [76]:
np.argmax(predictions)%6

1

In [77]:
##Takes the single highest prediction of the 9 segments
preds=[]
actuals=[]
for i in range(0,1000):
    lower = 9000+i*9
    upper = lower+9
    prediction = model.predict(seg_x[lower:upper])
    pred_label = np.argmax(prediction)%6
    preds.append(mode(pred_label,axis=None)[0][0])
    actuals.append(mode(seg_y[lower:upper],axis=None)[0][0])

In [78]:
conf_mx=confusion_matrix(actuals, preds)
print(conf_mx)

[[116   9   3   1  16   8]
 [ 13 150   1   0   6   0]
 [  7   5  91  12  41   2]
 [ 10  32  21  60  61   0]
 [  9   5  10  10 138   0]
 [112  11   0   2  23  15]]


In [79]:
print((116   +150   +91  +60  +138   +15)/len(preds))

0.57


In [80]:
##Sums down the column and then 
preds=[]
actuals=[]
for i in range(0,1000):
    lower = 9000+i*9
    upper = lower+9
    prediction = model.predict(seg_x[lower:upper])
    pred_label = np.argmax(np.sum(prediction,axis=0))
    preds.append(mode(pred_label,axis=None)[0][0])
    actuals.append(mode(seg_y[lower:upper],axis=None)[0][0])

In [81]:
conf_mx=confusion_matrix(actuals, preds)
print(conf_mx)

[[116   3   5   5  16   8]
 [ 10 148   1   4   5   2]
 [  2   4  80  21  51   0]
 [  3   6  21  95  59   0]
 [  3   0   8  11 150   0]
 [117   4   1   2  21  18]]


In [82]:
print((116   +148   +80  +95  +150   +18)/len(preds))

0.607


In [83]:
'''
CNN -> acc: 0.5568
Voting -> 0.595
Single Highest -> .57
Weighted Vote -> .607
'''

'\nCNN -> acc: 0.5568\nVoting -> 0.595\nSingle Highest -> .57\nWeighted Vote -> .607\n'